In [11]:
import pandas as pd
import datetime
from pathlib import Path
from tadpole.models import SimpleSVM
from tadpole.utils import test_train_split

tadpoleD1D2File = Path("../data/TADPOLE_D1_D2.csv")

df = pd.read_csv(tadpoleD1D2File)

# remove all rows without
df = df[df["Ventricles"].notnull()]
df = df[df["DXCHANGE"].notnull()]
df = df[df["ADAS13"].notnull()]

train_df, test_df, ground_truth_df = test_train_split(df, 0.1)

model = SimpleSVM()
model.train(train_df)

first_patient_id = test_df['RID'][0]
prediction = model.predict(test_df[test_df['RID'] == first_patient_id], datetime.datetime.now())
truth = ground_truth_df[ground_truth_df['RID'] == first_patient_id]

# evaluate (prediction, truth)

/home/tom/miniconda3/envs/tadpole/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (471,473,474,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,569,570,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,599,601,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,624,625,626,627,628,629,630,631,632,633,634,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,

In [5]:
def average_time_between_visits(df):
    df1 = df[['RID', 'EXAMDATE']].copy()
    df1 = df1.sort_values(by=['RID', 'EXAMDATE'])
    df1['EXAMDATE'] = df1['EXAMDATE'].map(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
    # Get future value from each row's next row
    df1["Future_EXAMDATE"] = df1['EXAMDATE'].shift(-1)
    # Drop each last row per patient
    df1 = df1.drop(df1.groupby('RID').tail(1).index.values)
    return (df1['Future_EXAMDATE']-df1['EXAMDATE']).mean()


In [7]:
average_time_between_visits(df)

Timedelta('207 days 07:53:03.860414')

In [10]:
truth

,RID,PTID,VISCODE,SITE,D1,D2,COLPROT,ORIGPROT,EXAMDATE,DX_bl,...,PHASE_UPENNBIOMK9_04_19_17,BATCH_UPENNBIOMK9_04_19_17,KIT_UPENNBIOMK9_04_19_17,STDS_UPENNBIOMK9_04_19_17,RUNDATE_UPENNBIOMK9_04_19_17,ABETA_UPENNBIOMK9_04_19_17,TAU_UPENNBIOMK9_04_19_17,PTAU_UPENNBIOMK9_04_19_17,COMMENT_UPENNBIOMK9_04_19_17,update_stamp_UPENNBIOMK9_04_19_17
0,3,011_S_0003,m24,11,1,0,ADNI1,ADNI1,2007-09-12,AD,...,,,,,,,,,,
